In [1]:
!rm Baby.sqlite
import pandas as pd
import pprint
import json
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float


In [2]:
#Pulling in the data#Pullin 
csv_path = "DataSets/SSA Baby Names by Decades.csv"

#Reading data
df = pd.read_csv(csv_path, encoding="utf-8", dtype=object)

In [3]:
df.head()

,Decade,Males,Number,Females,Number.1
0,1880,John,"89,950",Mary,"91,668"
1,1880,William,"84,881",Anna,"38,159"
2,1880,James,"54,056",Emma,"25,404"
3,1880,George,"47,651",Elizabeth,"25,006"
4,1880,Charles,"46,656",Margaret,"21,799"


In [4]:
#Types.set_index('Columns', inplace=True)
#Types = pd.DataFrame(list(df))
#Types.set_index(0, inplace=True)
#Types = Types.astype(str)
#Types.head()

In [5]:
df['id'] = df.index
#Types.set_index('Column', inplace=True)
df.head()

,Decade,Males,Number,Females,Number.1,id
0,1880,John,"89,950",Mary,"91,668",0
1,1880,William,"84,881",Anna,"38,159",1
2,1880,James,"54,056",Emma,"25,404",2
3,1880,George,"47,651",Elizabeth,"25,006",3
4,1880,Charles,"46,656",Margaret,"21,799",4


In [6]:
df = df[['id', 'Decade', 'Males', 'Number', 'Females', 'Number.1']]
df.head()

,id,Decade,Males,Number,Females,Number.1
0,0,1880,John,"89,950",Mary,"91,668"
1,1,1880,William,"84,881",Anna,"38,159"
2,2,1880,James,"54,056",Emma,"25,404"
3,3,1880,George,"47,651",Elizabeth,"25,006"
4,4,1880,Charles,"46,656",Margaret,"21,799"


In [7]:
df['Num'] = df['Number.1']
df = df.drop('Number.1', 1)
df.head()

,id,Decade,Males,Number,Females,Num
0,0,1880,John,"89,950",Mary,"91,668"
1,1,1880,William,"84,881",Anna,"38,159"
2,2,1880,James,"54,056",Emma,"25,404"
3,3,1880,George,"47,651",Elizabeth,"25,006"
4,4,1880,Charles,"46,656",Margaret,"21,799"


In [8]:
#df[['Decade']] = df[['Decade']].apply(pd.to_numeric)
#df.dtypes

In [9]:
engine = create_engine("sqlite:///Baby.sqlite")

In [10]:
conn = engine.connect()

In [11]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

Base = declarative_base()
class baby(Base):
    __tablename__ = 'Baby'

    id = Column(Integer, primary_key=True)
    Decade = Column(Integer)
    Males = Column(Text)
    Number = Column(Integer)
    Females = Column(Text)
    Num = Column(Text)
    


    def __repr__(self):
        return f"id={self.id}, name={self.name}"

Base.metadata.create_all(engine)

In [12]:
data = df.to_dict(orient='records')

In [13]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
table = sqlalchemy.Table('Baby', metadata, autoload=True)

In [15]:
conn.execute(table.delete())

In [16]:
conn.execute(table.insert(), data)

In [17]:
conn.execute("select * from Baby limit 5").fetchall()

[(0, 1880, 'John', '89,950', 'Mary', '91,668'),
 (1, 1880, 'William', '84,881', 'Anna', '38,159'),
 (2, 1880, 'James', '54,056', 'Emma', '25,404'),
 (3, 1880, 'George', '47,651', 'Elizabeth', '25,006'),
 (4, 1880, 'Charles', '46,656', 'Margaret', '21,799')]